In [35]:
import time
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.base import TransformerMixin
from sklearn_crfsuite import CRF
from sklearn.pipeline import Pipeline
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
import sklearn_crfsuite
from sklearn_crfsuite import metrics

In [36]:
class TokenFeatures(TransformerMixin):
    def transform(self, X, **transform_params):
        return [self.sent2features(s) for s in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def sent2features(self, sent):
        return [{ 'word.lower()': word.lower() } for word in sent.split()]

    def get_feature_names(self):
        return ['word.lower()']

In [37]:
pipeline = Pipeline([
    #('token_features', TokenFeatures()),  # Transform text to token-level features
    ('crf', CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100, all_possible_transitions=True))
])

In [38]:
def read_corpus_to_dataframe(filepath):
    data = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 2:
                token, label = parts
                data.append({'token': token, 'label': label})
                
    return pd.DataFrame(data)

In [39]:
df_train = read_corpus_to_dataframe('./data/es-en/train.conll')
df_dev = read_corpus_to_dataframe('./data/es-en/dev.conll')
#df_train.head()
df_train.tail()


,token,label
253216,callata,lang2
253217,borrega,lang2
253218,pelos,lang2
253219,necios,lang2
253220,!!!!!,other


In [40]:
X_train, y_train = df_train['token'], df_train['label']
X_dev, y_dev = df_dev['token'], df_dev['label']

In [41]:
crf = CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100, all_possible_transitions=True)
crf.fit(X_train, y_train)

ValueError: The numbers of items and labels differ: |x| = 4, |y| = 5

In [ ]:
X_train, y_train = df_train['token'], df_train['label']
X_dev, y_dev = df_dev['token'], df_dev['label']

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(min_df=1, max_df=0.5, ngram_range=(1,3))),
    ('sclf', LogisticRegression(random_state=42))
])

NameError: name 'LogisticRegression' is not defined